# Third Analysis

## Data preparation: 

#### Removing features, removing outliers, merging external data, scaling, doing PCA and implementing K-means optimal solution for the PCA of the selected features for the PVA dataset.

Using the Labs' code as guides we will adress the group project.

In [1]:
# Importing the libraries necessaries for the group project
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
import matplotlib.cm as cm
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.cluster import AgglomerativeClustering, KMeans
from scipy.cluster.hierarchy import dendrogram
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder

In [2]:
metric_features = ['age', '']

In [75]:
# Reading the dataset and first glimpse at the rows and columns
dataset_raw = pd.read_csv('donors.csv')
dataset_raw.head()
df = dataset_raw

C:\Users\fede_\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (9,412) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [76]:
len(dataset_raw)

95412

In [77]:
# Checking the name of the duplicated index and the using drop to have just one index
dataset_raw.columns
dataset_raw = dataset_raw.drop('Unnamed: 0', axis = 1)
dataset_raw.head()

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,AVGGIFT,CONTROLN,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
0,2009-01-01,GRI,0,IL,61081,,,1957-12-01,0,,...,7.741935,95515,0,L,4,E,X,X,X,C
1,2014-01-01,BOA,1,CA,91326,,,1972-02-01,0,,...,15.666667,148535,0,L,2,G,X,X,X,A
2,2010-01-01,AMH,1,NC,27017,,,NaN,0,,...,7.481481,15078,1,L,4,E,X,X,X,C
3,2007-01-01,BRY,0,CA,95953,,,1948-01-01,0,,...,6.812500,172556,1,L,4,E,X,X,X,C
4,2006-01-01,,0,FL,33176,,,1940-01-01,0,X,...,6.864865,7112,1,L,2,F,X,X,X,A


In [78]:
# Value counting each feature, diagnosing if it makes sense to maintain it.
# We are not going to change ODATEDW since it is the date of the first gift of the donor.
dataset_raw[dataset_raw.ODATEDW.isna()]

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,AVGGIFT,CONTROLN,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2


In [79]:
# Mailing list: we are going to check for unique values if there is any reasoning but most probably we are not going to 
#  hot encode the mailing lists into dummy variables in order to use at the clustering solution.
dataset_raw.OSOURCE.unique().shape

# Unrealistic to have 896 dummy variables or even have a variable that ranges from 0 to 895 encompassing the rationale 
#  of each of the possible mailing lists. Therefore we are going to discard this feature.
dataset_raw = dataset_raw.drop('OSOURCE', axis = 1)

In [80]:
# Same rationale as before a categorical feature that does not encompass that much information per se if you consider the amount
#  of adjustments you would need to do in order to aggregate that information into the clustering problem (hotencoding or 
#   considering a numerical variable). Therefore we are going to drop this columns as well.
dataset_raw.TCODE.unique().shape
dataset_raw = dataset_raw.drop('TCODE', axis = 1)

In [81]:
### STATE WON'T BE DROPPED AT LEAST FOR NOW SINCE WE ARE GOING TO USE IT THIS INFORMATION TO LATER MERGE A REPRESENTATION 
###  OF THE AVERAGE GDP PER CAPITA OF EACH STATE TO TRANSFORM THE CURRENT FEATURE INCOME IN A CONTINUOS VARIABLE. 

### DO THE SAME FOR WEALTH1 AND WEALTH2 USING US CENSUS BUREAU PAPER DATA ESTIMATED BY NETWORTH, HOME VALUE ETC...

In [82]:
### ZIP COULD BE HELPFUL LATER FOR GEOSPATIAL VISUALIZATION OF THE CLUSTERS, SO I WILL LEAVE IT AS IT IS FOR NOW AND THEN DECIDE IT

In [83]:
# MAILCODE won´t be helpful at all, not even for visualization purposes so will drop it as well.
dataset_raw = dataset_raw.drop('MAILCODE', axis = 1)

In [84]:
dataset_raw[dataset_raw.PVASTATE == 'E'].loc[:,['PVASTATE', 'STATE']].loc[:, 'STATE'].unique().shape

(3,)

In [85]:
#FDA 

dataset_raw[dataset_raw.PVASTATE == 'E']['STATE'].unique().shape 

# Only 3 states have E

(3,)

In [86]:
dataset_raw[dataset_raw.STATE == 'CA'].loc[:,['PVASTATE', 'STATE']].loc[:, 'PVASTATE'].unique()

array([' ', 'P'], dtype=object)

In [87]:
#FDA 
dataset_raw[dataset_raw['STATE'] == 'CA']['PVASTATE'].unique()

# CA example, it has nan and P

array([' ', 'P'], dtype=object)

In [88]:
# We just realize that there are rows that accounts for states that should have on the PVASTATE column the value P, 
#  and probably the same is happening fot states that should have the E value. 
# Therefore we are going to input the Ps and Es on these rows with respect to the corresponding state of that row already being 
#  associated in another row to that certain value for PVASTATE. 

# The array of states that should yield P in the PVASTATE column:
states_P_at_PVASTATE = dataset_raw[dataset_raw.PVASTATE == 'P'].loc[:,['PVASTATE', 'STATE']].loc[:, 'STATE'].unique()
# The array of states that should yield E in the PVASTATE column
states_E_at_PVASTATE = dataset_raw[dataset_raw.PVASTATE == 'E'].loc[:,['PVASTATE', 'STATE']].loc[:, 'STATE'].unique()


index_of_P = dataset_raw.loc[dataset_raw['STATE'].isin(states_P_at_PVASTATE)].loc[:, 'PVASTATE'].replace(" ", "P").index.tolist()
dataset_raw.loc[index_of_P, 'PVASTATE'] = 'P'
dataset_raw.loc[dataset_raw['STATE'].isin(states_P_at_PVASTATE)].loc[:,'PVASTATE'].unique()

index_of_E = dataset_raw.loc[dataset_raw['STATE'].isin(states_E_at_PVASTATE)].loc[:, 'PVASTATE'].replace(" ", "E").index.tolist()
dataset_raw.loc[index_of_E, 'PVASTATE'] = 'E'
dataset_raw.loc[dataset_raw['STATE'].isin(states_E_at_PVASTATE)].loc[:,'PVASTATE'].unique()

# We are going to drop this value since most of the records are from states that contains P value
# Later we can proceed to drop the 0.003 that accounts for nas and then we readress the analysis with just the Ps and Es 
#  transforming this feature into a dummy variable. Since it is just 0.3% of the sample we would not lost that much erasing these
#   records, although they account for about 20 states despite the low number of records
(dataset_raw.PVASTATE.value_counts()[1]/(dataset_raw.PVASTATE.value_counts().sum() - dataset_raw.PVASTATE.value_counts()[1]))

# From now on, we will call the dataset_raw without the PVASTATE feature db_raw:
db_raw = dataset_raw.drop('PVASTATE', axis = 1)

In [89]:
# Transforming into age for the analysis, we won't drop DOB for analysis purposes. Since we are trying to create something
#  that could easily be updated it with new records and reassessing the feature selection and engineering phases so it will 
#   not be encompassed into the clustering solution problem (a new feature called AGE will) but will stay present at the
#    final dataframe.

db_raw['age'] = db_raw['DOB'].str[:4]
db_raw['age'].replace(np.nan, 0, inplace = True)
db_raw['age'] = db_raw['age'].astype(int)
db_raw['age'] = 2020 - db_raw['age']
for i in db_raw['age']: 
    if db_raw.loc[i, 'age'] != 2020:
        db_raw.loc[i, 'age'] = None
db_raw['age'] = db_raw['age'].replace(2020, None)
db_raw['age']

0        63.0
1        48.0
2        48.0
3         NaN
4         NaN
         ... 
95407    52.0
95408    50.0
95409    62.0
95410    60.0
95411    82.0
Name: age, Length: 95412, dtype: float64

In [90]:
#FDA

db_raw['DOB_dt'] = pd.to_datetime(db_raw['DOB'])

now = pd.Timestamp('now')
db_raw['age_fda'] = (now - db_raw['DOB_dt']).astype('<m8[Y]')
db_raw[['DOB','DOB_dt','age','age_fda']]

,DOB,DOB_dt,age,age_fda
0,1957-12-01,1957-12-01,63.0,62.0
1,1972-02-01,1972-02-01,48.0,48.0
2,NaN,NaT,48.0,NaN
3,1948-01-01,1948-01-01,NaN,72.0
4,1940-01-01,1940-01-01,NaN,80.0
...,...,...,...,...
95407,NaN,NaT,52.0,NaN
95408,1970-01-01,1970-01-01,50.0,50.0
95409,1958-01-01,1958-01-01,62.0,62.0
95410,1960-05-01,1960-05-01,60.0,60.0


In [91]:
# We could runa KNN imputer using socio-demographic data at state level to put this age rows that are missing but since 
#  it is too little maybe it is not worth the trouble doind it at state level we could run the knn imputer at these features that
#   represent the family like age, income, wealth to avoid having to delete records. 

db_raw['age'].isna().value_counts()

False    95326
True        86
Name: age, dtype: int64

In [92]:
#FDA
db_raw['age_fda'].isna().value_counts()

False    71529
True     23883
Name: age_fda, dtype: int64

In [93]:
db_raw['age'].value_counts()

80.0    2576
50.0    2552
70.0    2537
52.0    2509
78.0    2474
        ... 
11.0       3
18.0       2
17.0       1
8.0        1
15.0       1
Name: age, Length: 87, dtype: int64

In [94]:
db_raw['age_fda'].value_counts()

80.0    1941
52.0    1919
70.0    1912
50.0    1893
78.0    1837
        ... 
7.0        2
15.0       1
8.0        1
11.0       1
17.0       1
Name: age_fda, Length: 86, dtype: int64

In [21]:
# This feature does not bring that much information (counting the number of times each value appears shows me that as well as other features 
#  does not need hot encoding or even keeping in the dataframe for further analysis later on).
db_raw.NOEXCH.value_counts()
db_raw = db_raw.drop('NOEXCH', axis = 1)

In [22]:
# I will transform these next few flags into dummy variables that can be used during the analysis further, after the clustering 
#  problem...

db_raw.RECINHSE = db_raw.RECINHSE.replace(" ", "0").replace("X", "1").astype(int)
db_raw.RECP3 = db_raw.RECP3.replace(" ", "0").replace("X", "1").astype(int)
db_raw.RECPGVG = db_raw.RECPGVG.replace(" ", "0").replace("X", "1").astype(int)
db_raw.RECSWEEP = db_raw.RECSWEEP.replace(" ", "0").replace("X", "1").astype(int)
db_raw

,ODATEDW,STATE,ZIP,DOB,RECINHSE,RECP3,RECPGVG,RECSWEEP,MDMAUD,DOMAIN,...,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2,age,DOB_dt,age_fda
0,2009-01-01,IL,61081,1957-12-01,0,0,0,0,XXXX,T2,...,L,4,E,X,X,X,C,63.0,1957-12-01,62.0
1,2014-01-01,CA,91326,1972-02-01,0,0,0,0,XXXX,S1,...,L,2,G,X,X,X,A,48.0,1972-02-01,48.0
2,2010-01-01,NC,27017,NaN,0,0,0,0,XXXX,R2,...,L,4,E,X,X,X,C,48.0,NaT,NaN
3,2007-01-01,CA,95953,1948-01-01,0,0,0,0,XXXX,R2,...,L,4,E,X,X,X,C,NaN,1948-01-01,72.0
4,2006-01-01,FL,33176,1940-01-01,1,1,0,0,XXXX,S2,...,L,2,F,X,X,X,A,NaN,1940-01-01,80.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,2016-01-01,AK,99504,NaN,0,0,0,0,XXXX,C2,...,L,1,G,X,X,X,C,52.0,NaT,NaN
95408,2016-01-01,TX,77379,1970-01-01,0,0,0,0,XXXX,C1,...,L,1,F,X,X,X,A,50.0,1970-01-01,50.0
95409,2015-01-01,MI,48910,1958-01-01,0,1,0,0,XXXX,C3,...,L,3,E,X,X,X,B,62.0,1958-01-01,62.0
95410,2006-01-01,CA,91320,1960-05-01,1,0,0,0,XXXX,C1,...,L,4,F,X,X,X,A,60.0,1960-05-01,60.0


In [23]:
# a dummy variable that flags for dormant donors(0s) and inactive, lapsed and current donors(1s)
db_raw['donor_flag'] = db_raw.MDMAUD 
db_raw['donor_flag']
values_to_flag_donor = db_raw.MDMAUD.value_counts()[1:].index.tolist()
index_of_donors = db_raw.loc[db_raw.donor_flag.isin(values_to_flag_donor)].index.tolist()
db_raw.loc[index_of_donors, 'donor_flag'] = 1
db_raw['donor_flag'] = db_raw['donor_flag'].replace(np.nan, 0)
db_raw

,ODATEDW,STATE,ZIP,DOB,RECINHSE,RECP3,RECPGVG,RECSWEEP,MDMAUD,DOMAIN,...,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2,age,DOB_dt,age_fda,donor_flag
0,2009-01-01,IL,61081,1957-12-01,0,0,0,0,XXXX,T2,...,4,E,X,X,X,C,63.0,1957-12-01,62.0,XXXX
1,2014-01-01,CA,91326,1972-02-01,0,0,0,0,XXXX,S1,...,2,G,X,X,X,A,48.0,1972-02-01,48.0,XXXX
2,2010-01-01,NC,27017,NaN,0,0,0,0,XXXX,R2,...,4,E,X,X,X,C,48.0,NaT,NaN,XXXX
3,2007-01-01,CA,95953,1948-01-01,0,0,0,0,XXXX,R2,...,4,E,X,X,X,C,NaN,1948-01-01,72.0,XXXX
4,2006-01-01,FL,33176,1940-01-01,1,1,0,0,XXXX,S2,...,2,F,X,X,X,A,NaN,1940-01-01,80.0,XXXX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,2016-01-01,AK,99504,NaN,0,0,0,0,XXXX,C2,...,1,G,X,X,X,C,52.0,NaT,NaN,XXXX
95408,2016-01-01,TX,77379,1970-01-01,0,0,0,0,XXXX,C1,...,1,F,X,X,X,A,50.0,1970-01-01,50.0,XXXX
95409,2015-01-01,MI,48910,1958-01-01,0,1,0,0,XXXX,C3,...,3,E,X,X,X,B,62.0,1958-01-01,62.0,XXXX
95410,2006-01-01,CA,91320,1960-05-01,1,0,0,0,XXXX,C1,...,4,F,X,X,X,A,60.0,1960-05-01,60.0,XXXX


In [24]:
#FDA

print(db_raw['donor_flag'].unique())

print(db_raw['MDMAUD'].unique())

display(db_raw[db_raw['donor_flag']==1])

# Error here? No 0's?

['XXXX' 1]
['XXXX' 'C1CM' 'D1CM' 'L1CM' 'C2CM' 'D2CM' 'I1CM' 'I2MM' 'I1LM' 'L2CM'
 'C1MM' 'C1LM' 'D5CM' 'I2CM' 'D5TM' 'L1LM' 'D5MM' 'C2MM' 'I5CM' 'C5CM'
 'I5MM' 'C2LM' 'C5MM' 'D2MM' 'L1MM' 'L2LM' 'C5TM' 'L2TM']


,ODATEDW,STATE,ZIP,DOB,RECINHSE,RECP3,RECPGVG,RECSWEEP,MDMAUD,DOMAIN,...,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2,age,DOB_dt,age_fda,donor_flag
107,2006-01-01,FL,32073,1958-12-01,1,0,0,0,C1CM,S2,...,1,G,C,1,C,B,62.0,1958-12-01,61.0,1
371,2008-01-01,TX,77511,1935-06-01,1,0,0,0,D1CM,T2,...,1,G,D,1,C,A,85.0,1935-06-01,85.0,1
477,2006-01-01,GA,30349,1950-01-01,1,0,0,0,L1CM,S2,...,1,G,L,1,C,A,70.0,1950-01-01,70.0,1
535,2010-01-01,MO,64633,2010-01-01,1,0,0,0,C2CM,R3,...,2,G,C,2,C,D,10.0,2010-01-01,10.0,1
618,2006-01-01,FL,33777,NaN,1,0,1,0,D2CM,S2,...,1,G,D,2,C,A,54.0,NaT,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94040,2011-01-01,CA,92115,1958-03-01,1,0,0,0,D1CM,U1,...,1,G,D,1,C,A,62.0,1958-03-01,62.0,1
94879,2006-01-01,SC,29205,1962-11-01,1,0,0,0,C1CM,C1,...,3,G,C,1,C,B,58.0,1962-11-01,58.0,1
95348,2006-01-01,CA,92545,1956-01-01,1,0,1,0,D2MM,C2,...,1,G,D,2,M,A,64.0,1956-01-01,64.0,1
95396,2011-01-01,CA,94305,1934-01-01,1,0,0,0,L2CM,S1,...,3,G,L,2,C,A,86.0,1934-01-01,86.0,1


In [25]:
# Now is time for checking for substrings that corresponds to recency, frequency and monetary at the MDMAUD
# Recency:
# Checking for current (C):
db_raw['current_dflag'] = 0
idx_curr_donors = db_raw[db_raw['MDMAUD'].str.startswith('C')].index.tolist()
db_raw.loc[idx_curr_donors, 'current_dflag'] = 1
# Checking for Lapsed (L):
db_raw['lapsed_dflag'] = 0
idx_lap_donors = db_raw[db_raw['MDMAUD'].str.startswith('L')].index.tolist()
db_raw.loc[idx_lap_donors, 'lapsed_dflag'] = 1
# Checking for Inactive (I):
db_raw['inactive_dflag'] = 0
idx_ina_donors = db_raw[db_raw['MDMAUD'].str.startswith('I')].index.tolist()
db_raw.loc[idx_ina_donors, 'inactive_dflag'] = 1

In [26]:
# Frequency:
# Checking for most frequent ones (5+ gifts):
db_raw['most_freq_dflag'] = 0
idx_most_freq = db_raw[db_raw['MDMAUD'].str.contains("5")].index.tolist()
db_raw.loc[idx_most_freq, 'most_freq_dflag'] = 1
# Checking for mid-level frequent ones (2 gifts):
db_raw['mid_freq_dflag'] = 0
idx_mid_freq = db_raw[db_raw['MDMAUD'].str.contains("2")].index.tolist()
db_raw.loc[idx_mid_freq, 'mid_freq_dflag'] = 1
# Checking for low frequent ones (1 gift):
db_raw['low_freq_dflag'] = 0
idx_low_freq = db_raw[db_raw['MDMAUD'].str.contains("1")].index.tolist()
db_raw.loc[idx_low_freq, 'low_freq_dflag'] = 1

In [27]:
# Monetary: 
# Deleting the last character since it is said to be disposable, and that could help on using endswith method 
#  to check for Monetary analysis (RFM last step of grouping).
#db_raw['MDMAUD'] = db_raw.apply(lambda x: x['MDMAUD'][:-1], axis = 1)
# Checking for the ones tha spent the most:
db_raw['top_spent_dflag'] = 0
idx_top_spent = db_raw[db_raw['MDMAUD'].str.endswith('T')].index.tolist()
db_raw.loc[idx_top_spent, 'top_spent_dflag'] = 1
# Checking for second best donors monetarily:
db_raw['major_spent_dflag'] = 0
idx_maj_spent = db_raw[db_raw['MDMAUD'].str.endswith('M')].index.tolist()
db_raw.loc[idx_maj_spent, 'major_spent_dflag'] = 1
# Checking for core (mid-level) donors monetarily:
db_raw['mid_spent_dflag'] = 0
idx_mid_spent = db_raw[db_raw['MDMAUD'].str.endswith('C')].index.tolist()
db_raw.loc[idx_mid_spent, 'mid_spent_dflag'] = 1
db_raw['mid_spent_dflag'].value_counts()
# Checking for low-level donors monetarily:
db_raw['low_spent_dflag'] = 0
idx_low_spent = db_raw[db_raw['MDMAUD'].str.endswith('L')].index.tolist()
db_raw.loc[idx_low_spent, 'low_spent_dflag'] = 1
db_raw['low_spent_dflag'].value_counts()

### LATER, WE CAN TRANSFORM THIS INTO THREE COLUMNS WITH 5-3-3 RFM ANALYSIS FOR GROUPBY LATER...

0    95412
Name: low_spent_dflag, dtype: int64

In [28]:
#idx_non_curr_donors = db_raw[db_raw['current_dflag'].str.contains('C')].index.tolist()
#db_raw[idx_non_curr_donors]

# Error here?

AttributeError: Can only use .str accessor with string values!

In [30]:
# Using a for loop to enumerate each and every datatype in each column
for i, v in enumerate(dataset_raw.dtypes):
    print("\ncol.", i, ":  ", dataset_raw.columns[i], v, "\n")

# Why dataset_raw and not db_raw?     


col. 0 :   ODATEDW object 


col. 1 :   STATE object 


col. 2 :   ZIP object 


col. 3 :   PVASTATE object 


col. 4 :   DOB object 


col. 5 :   NOEXCH object 


col. 6 :   RECINHSE object 


col. 7 :   RECP3 object 


col. 8 :   RECPGVG object 


col. 9 :   RECSWEEP object 


col. 10 :   MDMAUD object 


col. 11 :   DOMAIN object 


col. 12 :   HOMEOWNR object 


col. 13 :   CHILD03 object 


col. 14 :   CHILD07 object 


col. 15 :   CHILD12 object 


col. 16 :   CHILD18 object 


col. 17 :   NUMCHLD float64 


col. 18 :   INCOME float64 


col. 19 :   GENDER object 


col. 20 :   WEALTH1 float64 


col. 21 :   HIT int64 


col. 22 :   MBCRAFT float64 


col. 23 :   MBGARDEN float64 


col. 24 :   MBBOOKS float64 


col. 25 :   MBCOLECT float64 


col. 26 :   MAGFAML float64 


col. 27 :   MAGFEM float64 


col. 28 :   MAGMALE float64 


col. 29 :   PUBGARDN float64 


col. 30 :   PUBCULIN float64 


col. 31 :   PUBHLTH float64 


col. 32 :   PUBDOITY float64 


col. 33 :   PUBNEWF

col. 376 :   ADATE_23 object 


col. 377 :   ADATE_24 object 


col. 378 :   RFA_2 object 


col. 379 :   RFA_3 object 


col. 380 :   RFA_4 object 


col. 381 :   RFA_5 object 


col. 382 :   RFA_6 object 


col. 383 :   RFA_7 object 


col. 384 :   RFA_8 object 


col. 385 :   RFA_9 object 


col. 386 :   RFA_10 object 


col. 387 :   RFA_11 object 


col. 388 :   RFA_12 object 


col. 389 :   RFA_13 object 


col. 390 :   RFA_14 object 


col. 391 :   RFA_15 object 


col. 392 :   RFA_16 object 


col. 393 :   RFA_17 object 


col. 394 :   RFA_18 object 


col. 395 :   RFA_19 object 


col. 396 :   RFA_20 object 


col. 397 :   RFA_21 object 


col. 398 :   RFA_22 object 


col. 399 :   RFA_23 object 


col. 400 :   RFA_24 object 


col. 401 :   CARDPROM int64 


col. 402 :   MAXADATE object 


col. 403 :   NUMPROM int64 


col. 404 :   CARDPM12 int64 


col. 405 :   NUMPRM12 int64 


col. 406 :   RDATE_3 object 


col. 407 :   RDATE_4 object 


col. 408 :   RDATE_5 object 


col. 4

In [33]:
#FDA

db_raw.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95412 entries, 0 to 95411
Data columns (total 484 columns):
 #   Column             Dtype         
---  ------             -----         
 0   ODATEDW            object        
 1   STATE              object        
 2   ZIP                object        
 3   DOB                object        
 4   RECINHSE           int32         
 5   RECP3              int32         
 6   RECPGVG            int32         
 7   RECSWEEP           int32         
 8   MDMAUD             object        
 9   DOMAIN             object        
 10  HOMEOWNR           object        
 11  CHILD03            object        
 12  CHILD07            object        
 13  CHILD12            object        
 14  CHILD18            object        
 15  NUMCHLD            float64       
 16  INCOME             float64       
 17  GENDER             object        
 18  WEALTH1            float64       
 19  HIT                int64         
 20  MBCRAFT            float64 

In [35]:
# Checking for NAs and using a loop to showcase each and every count of nas for every column.
for i, v in enumerate(dataset_raw.isna().sum()):
    if v != 0:
        print("\ncol.", i, ":  ",dataset_raw.columns[i], v,"\n")
        
# Why dataset_raw and not db_raw?  


col. 4 :   DOB 23883 


col. 17 :   NUMCHLD 83026 


col. 18 :   INCOME 21286 


col. 20 :   WEALTH1 44732 


col. 22 :   MBCRAFT 52854 


col. 23 :   MBGARDEN 52854 


col. 24 :   MBBOOKS 52854 


col. 25 :   MBCOLECT 52914 


col. 26 :   MAGFAML 52854 


col. 27 :   MAGFEM 52854 


col. 28 :   MAGMALE 52854 


col. 29 :   PUBGARDN 52854 


col. 30 :   PUBCULIN 52854 


col. 31 :   PUBHLTH 52854 


col. 32 :   PUBDOITY 52854 


col. 33 :   PUBNEWFN 52854 


col. 34 :   PUBPHOTO 52854 


col. 35 :   PUBOPP 52854 


col. 47 :   WEALTH2 43823 


col. 189 :   MSA 132 


col. 190 :   ADI 132 


col. 191 :   DMA 132 


col. 356 :   ADATE_3 1950 


col. 357 :   ADATE_4 2191 


col. 358 :   ADATE_5 33590 


col. 359 :   ADATE_6 3557 


col. 360 :   ADATE_7 8874 


col. 361 :   ADATE_8 3511 


col. 362 :   ADATE_9 11245 


col. 363 :   ADATE_10 32748 


col. 364 :   ADATE_11 10422 


col. 365 :   ADATE_12 8923 


col. 366 :   ADATE_13 40219 


col. 367 :   ADATE_14 18867 


col. 368 :   ADATE

In [72]:
# FDA
#pd.set_option("display.max_rows", None, "display.max_columns", None)
#pd.reset_option('all')

nans = pd.DataFrame(dataset_raw.isna().sum())
display(nans[nans[0]>0])

,0
DOB,23883
NUMCHLD,83026
INCOME,21286
WEALTH1,44732
MBCRAFT,52854
...,...
RAMNT_24,77674
FISTDATE,2
NEXTDATE,9973
TIMELAG,9973


In [43]:
# Checking duplicated rows.
dataset_raw.duplicated().sum()

0

In [44]:
dataset_raw.describe(include = "all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
ODATEDW,95412,54,2015-01-01,15358,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STATE,95412,57,CA,17343,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZIP,95412,19938,85351,61,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PVASTATE,95412,3,P,94924,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DOB,71529,847,1968-01-01,1479,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NOEXCH,95412,6,0,61203,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RECINHSE,95412,2,,88709,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RECP3,95412,2,,93395,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RECPGVG,95412,2,,95298,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RECSWEEP,95412,2,,93795,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
dataset_raw.describe().T

,count,mean,std,min,25%,50%,75%,max
NUMCHLD,12386.0,1.527773,0.806861,1.000000,1.000000,1.000000,2.000000,7.0
INCOME,74126.0,3.886248,1.854960,1.000000,2.000000,4.000000,5.000000,7.0
WEALTH1,50680.0,5.345699,2.742490,0.000000,3.000000,6.000000,8.000000,9.0
HIT,95412.0,3.321438,9.306899,0.000000,0.000000,0.000000,3.000000,241.0
MBCRAFT,42558.0,0.152075,0.470023,0.000000,0.000000,0.000000,0.000000,6.0
MBGARDEN,42558.0,0.059166,0.262078,0.000000,0.000000,0.000000,0.000000,4.0
MBBOOKS,42558.0,1.116382,1.669967,0.000000,0.000000,0.000000,2.000000,9.0
MBCOLECT,42498.0,0.063932,0.297344,0.000000,0.000000,0.000000,0.000000,6.0
MAGFAML,42558.0,0.448329,0.821010,0.000000,0.000000,0.000000,1.000000,9.0
MAGFEM,42558.0,0.126322,0.380916,0.000000,0.000000,0.000000,0.000000,5.0


In [ ]:
len(dataset_raw.isna().sum().index)

In [ ]:
dataset_raw.isna().sum() != 0

In [ ]:
# Single Metric Variable Histogram
plt.hist(dataset_raw[dataset_raw['TIMELAG'] < 40]['TIMELAG'], bins = 300)  # mess around with the bins
plt.title('TIMELAG', y = -0.2)

plt.show()

In [ ]:
dataset_raw[dataset_raw['TIMELAG'] < 40]

In [ ]:
dataset_raw.shape

In [ ]:
dataset_raw['ODATEDW'].unique()

In [ ]:
dataset_raw['STATE'].unique()

In [ ]:
dataset_raw['age'] = dataset_raw['DOB'].str[:4]

dataset_raw['age'].replace(np.nan, 0, inplace = True)

dataset_raw['age'] = dataset_raw['age'].astype(int)
dataset_raw['age'] = 2020 - dataset_raw['age']



In [ ]:
for i in dataset_raw['age']: 
    if dataset_raw['age'][i] != 2020:
        dataset_raw['age'][i] = None

In [ ]:
data_wo_agena = dataset_raw[dataset_raw["age"] != 2020]

In [ ]:
data_wo_agena.loc[:,["NUMCHLD", "INCOME", "WEALTH1", "WEALTH2", "age"]]
data_wo_agena.loc[:,["age"]].corr(data_wo_agena)

In [ ]:
dataset_raw.corr()

In [ ]:
dsetraw_neighbors = dataset_raw.copy()

In [ ]:
nans_idx = dsetraw_neighbors.isna().any(axis = 1)
dsetraw_neighbors[nans_idx]

In [ ]:
dataset_raw.iloc[:,20:].dtypes

In [ ]:
dataset_raw.iloc[:,22].value_counts()

In [ ]:
from math import ceil

# All Numeric Variables' Box Plots in one figure
sns.set()

# Prepare figure. Create individual axes where each box plot will be placed
fig, axes = plt.subplots(35, ceil(len(dataset_raw.select_dtypes(np.number).columns.tolist()) / 35), figsize = (200, 100))

# Plot data
# Iterate across axes objects and associate each box plot (hint: use the ax argument):
for ax, feat in zip(axes.flatten(), dataset_raw.select_dtypes(np.number).columns.tolist()): # Notice the zip() function and flatten() method
    sns.boxplot(dataset_raw[feat], ax=ax)
    
# Layout
# Add a centered title to the figure:
title = "Numeric Variables' Box Plots"

plt.suptitle(title)

plt.show()

In [ ]:
dataset_raw.select_dtypes(np.number).columns.tolist()

In [ ]:
from math import ceil

# All Numeric Variables' Box Plots in one figure
sns.set()

# Prepare figure. Create individual axes where each box plot will be placed
fig, axes = plt.subplots(7, ceil(len(dataset_raw.select_dtypes(np.number).columns.tolist()[:34]) / 7), figsize = (20, 11))

# Plot data
# Iterate across axes objects and associate each box plot (hint: use the ax argument):
for ax, feat in zip(axes.flatten(), dataset_raw.select_dtypes(np.number).columns.tolist()[:34]): # Notice the zip() function and flatten() method
    sns.boxplot(dataset_raw[feat], ax=ax)
    ax.set_title(feat, y=-0.13)

# Layout
# Add a centered title to the figure:
title = "Numeric Variables' Box Plots 0 to 34"

plt.suptitle(title)

plt.show()

In [ ]:
#DROP
TCODE


In [ ]:
# DROP THESE
dataset_raw[dataset_raw['HIT'] > 200]

In [ ]:
sns.distplot(dataset_raw['INCOME'])

In [ ]:
sns.distplot(dataset_raw[dataset_raw['STATE'] == 'AL']['INCOME'])

In [ ]:
# Create a Income variable contigent on 
dataset_raw['STATE'].unique()

In [ ]:
dataset_raw.select_dtypes(np.number).columns.tolist()

In [ ]:
dataset_raw.select_dtypes(np.number)

In [ ]:
df_central = dataset_raw.select_dtypes(np.number).copy()

In [ ]:
df_central.fillna(df_central.median(), inplace=True)

In [ ]:
df_central.isna().sum() 

In [ ]:
df_central = df_central.drop('TCODE', axis = 1)

In [ ]:
pca = PCA()
pca_feat = pca.fit_transform(df_central)

In [ ]:
# figure and axes
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# draw plots
ax1.plot(pca.explained_variance_, marker=".", markersize=12)
ax2.plot(pca.explained_variance_ratio_, marker=".", markersize=12, label="Proportion")
ax2.plot(np.cumsum(pca.explained_variance_ratio_), marker=".", markersize=12, linestyle="--", label="Cumulative")

# customizations
ax2.legend()
ax1.set_title("Scree Plot", fontsize=14)
ax2.set_title("Variance Explained", fontsize=14)
ax1.set_ylabel("Eigenvalue")
ax2.set_ylabel("Proportion")
ax1.set_xlabel("Components")
ax2.set_xlabel("Components")
ax1.set_xticks(range(0, pca.n_components_, 2))
ax1.set_xticklabels(range(1, pca.n_components_ + 1, 2))
ax2.set_xticks(range(0, pca.n_components_, 2))
ax2.set_xticklabels(range(1, pca.n_components_ + 1, 2))

plt.show()

In [ ]:
# Output PCA table
pd.DataFrame(
    {"Eigenvalue": pca.explained_variance_,
     "Difference": np.insert(np.diff(pca.explained_variance_), 0, 0),
     "Proportion": pca.explained_variance_ratio_,
     "Cumulative": np.cumsum(pca.explained_variance_ratio_)},
    index=range(1, pca.n_components_ + 1)
)[0:10]

In [ ]:
# figure and axes
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(30, 10))

# draw plots
ax1.plot(pca.explained_variance_, marker=".", markersize=12)
ax2.plot(pca.explained_variance_ratio_, marker=".", markersize=12, label="Proportion")
ax2.plot(np.cumsum(pca.explained_variance_ratio_), marker=".", markersize=12, linestyle="--", label="Cumulative")

# customizations
ax2.legend()
ax1.set_title("Scree Plot", fontsize=14)
ax2.set_title("Variance Explained", fontsize=14)
ax1.set_ylabel("Eigenvalue")
ax2.set_ylabel("Proportion")
ax1.set_xlabel("Components")
ax2.set_xlabel("Components")
ax1.set_xticks(range(0, pca.n_components_, 2))
ax1.set_xticklabels(range(1, pca.n_components_ + 1, 2))
ax2.set_xticks(range(0, pca.n_components_, 2))
ax2.set_xticklabels(range(1, pca.n_components_ + 1, 2))

plt.show()

In [ ]:
pca_scaled = PCA(n_components=4)
pca_feat_scaled = pca_scaled.fit_transform(df_central_scaled)
pca_feat_names_scaled = [f"PC{i}" for i in range(pca_scaled.n_components_)]
pca_df_scaled = pd.DataFrame(pca_feat_scaled, index=df_central_scaled.index, columns=pca_feat_names_scaled)  # remember index=df_pca.index
pca_df_scaled

In [ ]:
df_central_scaled = df_central
df_central_scaled

In [ ]:
scaler = MinMaxScaler()
scaled_feat = scaler.fit_transform(df_central_scaled)
df_central_scaled = scaled_feat

In [ ]:
pca_scaled = PCA()
pca_feat_scaled = pca_scaled.fit_transform(df_central_scaled)

In [ ]:
# Output PCA table
pd.DataFrame(
    {"Eigenvalue": pca_scaled.explained_variance_,
     "Difference": np.insert(np.diff(pca_scaled.explained_variance_), 0, 0),
     "Proportion": pca_scaled.explained_variance_ratio_,
     "Cumulative": np.cumsum(pca_scaled.explained_variance_ratio_)},
    index=range(1, pca_scaled.n_components_ + 1)
)[:50]

In [ ]:
pca_scaled = PCA(n_components=20)
pca_feat_scaled = pca_scaled.fit_transform(df_central_scaled)
pca_feat_names_scaled = [f"PC{i}" for i in range(pca_scaled.n_components_)]
pca_df_scaled = pd.DataFrame(pca_feat_scaled, index=pd.DataFrame(df_central_scaled).index, columns=pca_feat_names_scaled)  # remember index=df_pca.index
pca_df_scaled

In [ ]:
# Reassigning df to contain pca variables
df_central_scaled = pd.concat([pd.DataFrame(df_central_scaled), pca_df_scaled], axis=1)
df_central_scaled.head()

In [ ]:
# def _color_red_or_green(val):
#    if val < -0.45:
#        color = 'background-color: red'
#    elif val > 0.45:
#        color = 'background-color: green'
#    else:
#        color = ''
#    return color

# Interpreting each Principal Component
#loadings = df_central_scaled.corr()
#loadings = loadings.style.applymap(_color_red_or_green)
loadings = loadings.iloc[:,-20:]
loadings[loadings.round(2) > 0.1]

In [ ]:
range_clusters = range(6, 30)

In [ ]:
inertia = []
for n_clus in range_clusters:  # iterate over desired ncluster range
    kmclust = KMeans(n_clusters=n_clus, init='k-means++', n_init=15, random_state=1)
    kmclust.fit(df_central_scaled)
    inertia.append(kmclust.inertia_)  # save the inertia of the given cluster solution

In [ ]:
# The inertia plot
plt.figure(figsize=(9,5))
plt.plot(inertia)
plt.ylabel("Inertia: SSw")
plt.xlabel("Number of clusters")
plt.title("Inertia plot over clusters", size=15)
plt.show()

In [ ]:
# The inertia plot
plt.figure(figsize=(9,5))
plt.plot(inertia)
plt.ylabel("Inertia: SSw")
plt.xlabel("Number of clusters")
plt.title("Inertia plot over clusters", size=15)
plt.show()

In [ ]:
# Adapted from:
# https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html#sphx-glr-auto-examples-cluster-plot-kmeans-silhouette-analysis-py

# Storing average silhouette metric
avg_silhouette = []
for nclus in range_clusters:
    # Skip nclus == 1
    if nclus == 1:
        continue
    
    # Create a figure
    fig = plt.figure(figsize=(13, 7))

    # Initialize the KMeans object with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    kmclust = KMeans(n_clusters=nclus, init='k-means++', n_init=15, random_state=1)
    cluster_labels = kmclust.fit_predict(df_central)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed clusters
    silhouette_avg = silhouette_score(df_central, cluster_labels)
    avg_silhouette.append(silhouette_avg)
    print(f"For n_clusters = {nclus}, the average silhouette_score is : {silhouette_avg}")

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(df_central, cluster_labels)

    y_lower = 10
    for i in range(nclus):
        # Aggregate the silhouette scores for samples belonging to cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]
        ith_cluster_silhouette_values.sort()
        
        # Get y_upper to demarcate silhouette y range size
        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i
        
        # Filling the silhouette
        color = cm.nipy_spectral(float(i) / nclus)
        plt.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        plt.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    plt.title("The silhouette plot for the various clusters.")
    plt.xlabel("The silhouette coefficient values")
    plt.ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    plt.axvline(x=silhouette_avg, color="red", linestyle="--")
    
    # The silhouette coefficient can range from -1, 1
    xmin, xmax = np.round(sample_silhouette_values.min() -0.1, 2), np.round(sample_silhouette_values.max() + 0.1, 2)
    plt.xlim([xmin, xmax])
    
    # The (nclus+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    plt.ylim([0, len(df_central) + (nclus + 1) * 10])

    plt.yticks([])  # Clear the yaxis labels / ticks
    plt.xticks(np.arange(xmin, xmax, 0.1))

In [ ]:
# The average silhouette plot
# The inertia plot
plt.figure(figsize=(9,5))
plt.plot(avg_silhouette)
plt.ylabel("Average silhouette")
plt.xlabel("Number of clusters")
plt.title("Average silhouette plot over clusters", size=15)
plt.show()

In [ ]:
hclust = AgglomerativeClustering(linkage='ward', affinity='euclidean', n_clusters=5)
hc_labels = hclust.fit_predict(pca_df)

In [ ]:
# Computing SST
X = df_central.values
sst = np.sum(np.square(X - X.mean(axis=0)), axis=0)

# Computing SSW
ssw_iter = []
for i in np.unique(hc_labels):
    X_k = X[hc_labels == i]
    ssw_iter.append(np.sum(np.square(X_k - X_k.mean(axis=0)), axis=0))
ssw = np.sum(ssw_iter, axis=0)

# Computing SSB
ssb_iter = []
for i in np.unique(hc_labels):
    X_k = X[hc_labels == i]
    ssb_iter.append(X_k.shape[0] * np.square(X_k.mean(axis=0) - X.mean(axis=0)))
ssb = np.sum(ssb_iter, axis=0)

# Verifying the formula
np.round(sst) == np.round((ssw + ssb))

In [ ]:
def get_r2_hc(df, link_method, max_nclus, min_nclus=1, dist="euclidean"):
    """This function computes the R2 for a set of cluster solutions given by the application of a hierarchical method.
    The R2 is a measure of the homogenity of a cluster solution. It is based on SSt = SSw + SSb and R2 = SSb/SSt. 
    
    Parameters:
    df (DataFrame): Dataset to apply clustering
    link_method (str): either "ward", "complete", "average", "single"
    max_nclus (int): maximum number of clusters to compare the methods
    min_nclus (int): minimum number of clusters to compare the methods. Defaults to 1.
    dist (str): distance to use to compute the clustering solution. Must be a valid distance. Defaults to "euclidean".
    
    Returns:
    ndarray: R2 values for the range of cluster solutions
    """
    def get_ss(df):
        ss = np.sum(df.var() * (df.count() - 1))
        return ss  # return sum of sum of squares of each df variable
    
    sst = get_ss(df)  # get total sum of squares
    
    r2 = []  # where we will store the R2 metrics for each cluster solution
    
    for i in range(min_nclus, max_nclus+1):  # iterate over desired ncluster range
        cluster = AgglomerativeClustering(n_clusters=i, affinity=dist, linkage=link_method)
        hclabels = cluster.fit_predict(df) #get cluster labels
        df_concat = pd.concat((df, pd.Series(hclabels, name='labels')), axis=1)  # concat df with labels
        ssw_labels = df_concat.groupby(by='labels').apply(get_ss)  # compute ssw for each cluster labels
        ssb = sst - np.sum(ssw_labels)  # remember: SST = SSW + SSB
        r2.append(ssb / sst)  # save the R2 of the given cluster solution
        
    return np.array(r2)

In [ ]:
# Prepare input
hc_methods = ["ward", "complete", "average", "single"]
# Call function defined above to obtain the R2 statistic for each hc_method
max_nclus = 10
r2_hc_methods = np.vstack([get_r2_hc(df=df_central, link_method=link, max_nclus=max_nclus) for link in hc_methods]).T
r2_hc_methods = pd.DataFrame(r2_hc_methods, index=range(1, max_nclus + 1), columns=hc_methods)

sns.set()
# Plot data
fig = plt.figure(figsize=(11,5))
sns.lineplot(data=r2_hc_methods, linewidth=2.5, markers=["o"]*4)

# Finalize the plot
fig.suptitle("R2 plot for various hierarchical methods", fontsize=21)
plt.gca().invert_xaxis()  # invert x axis
plt.legend(title="HC methods", title_fontsize=11)
plt.xticks(range(1, max_nclus + 1))
plt.xlabel("Number of clusters", fontsize=13)
plt.ylabel("R2 metric", fontsize=13)

plt.show()